In [15]:
# load dataset
import pickle
train_set = pickle.load(open('data/alfred_data/train_text_with_ppdl_low_appended.p', 'rb'))
val_set_seen = pickle.load(open('data/alfred_data/val_seen_text_with_ppdl_low_appended.p', 'rb'))
val_set_unseen = pickle.load(open('data/alfred_data/val_unseen_text_with_ppdl_low_appended.p', 'rb'))
obj2idx = pickle.load(open('data/alfred_data/alfred_dicts/obj2idx.p', 'rb'))
label_arg = 'object_targets'; num_labels = len(obj2idx)

In [7]:
import wandb
wandb.login()

True

In [8]:
sweep_config = {
    'method': 'grid',
}

In [9]:
parameters_dict = {
    'epoch': {
        'values': [20, 25, 35, 40, 50]
        },
    'learning_rate': {
        'values': [0.0001, 0.0005, 0.001]
        },
    'decay': {
        'values': [0.5, 0.6, 0.7, 0.8]
        },
    'decay_term': {
          'values': [5, 6, 7, 8, 9, 10]
        },
    }

sweep_config['parameters'] = parameters_dict

In [10]:
import pprint
pprint.pprint(sweep_config)

{'method': 'grid',
 'parameters': {'decay': {'values': [0.5, 0.6, 0.7, 0.8]},
                'decay_term': {'values': [5, 6, 7, 8, 9, 10]},
                'epoch': {'values': [20, 25, 35, 40, 50]},
                'learning_rate': {'values': [0.0001, 0.0005, 0.001]}}}


In [11]:
sweep_id = wandb.sweep(sweep_config, project="wandb-sweeps-demo")

Create sweep with ID: 0d9uy30p
Sweep URL: https://wandb.ai/chaozhoulim/wandb-sweeps-demo/sweeps/0d9uy30p


In [12]:
import torch

#Set device
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [13]:
from transformers import BertTokenizer as Tokenizer
from transformers import BertForSequenceClassification as Model
tokenizer = Tokenizer.from_pretrained('bert-base-uncased')
model = Model.from_pretrained('bert-base-uncased', num_labels=num_labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
import random
from random import shuffle
import numpy as np
x_train = train_set['x_low']; y_train = train_set['y']

x_train = [str(y_train[i])+ ' ' + x for i, x in enumerate(x_train)] 

x_ones = [i for i, lab in enumerate(y_train) if lab == 1]
random.seed(0)
shuffled = [i for i in x_ones]
shuffle(shuffled)
#x_train = np.array(train_set['x_low'])[shuffled]
x_train = np.array(x_train)[shuffled]
x_train = x_train.tolist()
encoding = tokenizer(x_train, return_tensors='pt', padding=True, truncation=True)
input_ids = encoding['input_ids'].to(device) #has shape [21025, 37]
attention_mask = encoding['attention_mask'].to(device) #also has shape [21025, 37]

label_train = np.array(train_set[label_arg])[shuffled]
label_train = torch.tensor(label_train).to(device)

In [16]:
x_val_seen = val_set_seen['x_low']; y_val_seen = val_set_seen['y']
x_val_seen = [str(y_val_seen[i])+ ' ' + x for i, x in enumerate(x_val_seen)]

x_ones_vs = [i for i, lab in enumerate(y_val_seen) if lab == 1]
x_val_seen = np.array(x_val_seen)[x_ones_vs].tolist()
encoding_v_s = tokenizer(x_val_seen, return_tensors='pt', padding=True, truncation=True)
input_ids_val_seen = encoding_v_s['input_ids'].to(device)
attention_mask_val_seen = encoding_v_s['attention_mask'].to(device) 

label_val_seen = [val_set_seen[label_arg][i] for i in x_ones_vs]
label_val_seen = torch.tensor(label_val_seen).to(device)

In [ ]:
with wandb.init(config=None):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        loader = build_dataset(config.batch_size)
        network = build_network(config.fc_layer_size, config.dropout)
        optimizer = build_optimizer(network, config.optimizer, config.learning_rate)

        for epoch in range(config.epochs):
            avg_loss = train_epoch(network, loader, optimizer)
            wandb.log({"loss": avg_loss, "epoch": epoch})        